In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import json
import os
root = './data/checkpoint/'
dataset = "cifar100"
agg = "lowrank"
alpha = 'alpha0'
ratio = 'ratio0.5'
warmup = 'warmup0'
dynamic='dynamicFalse'
dir_name = '_'.join([agg,alpha,ratio,warmup,dynamic])#,warmup,unit
print(dir_name)

model = "resnet34_2:resnet34_4:resnet34_8:resnet34_12"
file_list = sorted(os.listdir(os.path.join(root,dataset,model,dir_name)))
checkpoint_file = open(os.path.join(root,dataset,model,dir_name,file_list[-1],'0/log-1.json'))
hete_lowrank_data = json.load(checkpoint_file)
print(hete_lowrank_data)

In [ ]:
lowrank_model = "resnet34_12"
agg = "fedavg"
dir_name = '_'.join([agg,alpha,ratio,warmup,dynamic])#,warmup,unit

file_list = sorted(os.listdir(os.path.join(root,dataset,lowrank_model,dir_name)))
checkpoint_file = open(os.path.join(root,dataset,lowrank_model,dir_name,file_list[-1],'0/log-1.json'))
homo_lowrank_data = json.load(checkpoint_file)
print(homo_lowrank_data)

In [ ]:
pruning_model = "resnet34_1:resnet34_0.64:resnet34_0.5:resnet34_0.4"
agg = "pruning"
dir_name = '_'.join([agg,alpha,ratio,warmup,dynamic])#,warmup,unit

file_list = sorted(os.listdir(os.path.join(root,dataset,pruning_model,dir_name)))
checkpoint_file = open(os.path.join(root,dataset,pruning_model,dir_name,file_list[-1],'0/log-1.json'))
hete_pruning_data = json.load(checkpoint_file)
print(hete_pruning_data)

In [ ]:
import matplotlib.pyplot as plt
pruning_target_name = 'aggregated_test_loader_resnet34_0.4-0'
lowrank_target_name = 'aggregated_test_loader_resnet34_12-0'
large_target_name = 'aggregated_test_loader-0'

client_num = 10
linestyles =['--','-','-.']
def draw_movement(path,labels,names,file_datas,params):
    max_acc = 0
    plt.figure(dpi=300,figsize=(8,6))
    
    for label,name, datas,param,linestyle in zip(labels,names,file_datas,params,linestyles):
        y = []
        comm_cost = 0
        for i,data in enumerate(datas):
            if data['type'] == name:
                y.append(data['top1'])
                    
        x = np.array(list(range(len(y))))
        
        max_acc = max(np.max(y) + 1, max_acc)
        sns.lineplot(
            x = x ,
            y = y,
            legend = 'brief',
            label = label,
            ci = "sd",
            linewidth=2,
            linestyle=linestyle
        )
    ax = plt.gca()
    x_major_locator=MultipleLocator(25)
    ax.xaxis.set_major_locator(x_major_locator)
    plt.grid()
    plt.legend(fontsize=25)
    plt.ylabel('Top-1 test accuracy',fontsize=30)
    plt.xlabel('# of communication rounds',fontsize=30)
    min_acc = 20
    plt.ylim(min_acc, max_acc)
    plt.tick_params(labelsize=30)
    plt.savefig(path,bbox_inches='tight')
    plt.show()
    
draw_movement('./pictures/cifar100_compare_iid.pdf',
              ['HeteroFL','FedHM','Low Rank Model'],\
              [pruning_target_name,lowrank_target_name,large_target_name],\
              [hete_pruning_data,hete_lowrank_data,homo_lowrank_data],\
              [3.42,2.71,2.71])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from matplotlib.pyplot import MultipleLocator
import os
root = './data/checkpoint/'
dataset = "cifar100"
model = "resnet34"
fullrank_model = "_".join([model,"1"])
agg = "fedavg"
alpha = 'alpha0'
ratio = 'ratio0.5'

warmup = 'warmup0
dynamic='dynamicFalse'
dir_name = '_'.join([agg,alpha,ratio,warmup,dynamic])#,warmup,unit
print(dir_name)

dir_name = '_'.join([agg,alpha,ratio,warmup,dynamic])#,warmup,
print(dir_name)

file_list = sorted(os.listdir(os.path.join(root,dataset,fullrank_model,dir_name)))
checkpoint_file = open(os.path.join(root,dataset,fullrank_model,dir_name,file_list[-1],'0/log-1.json'))
fullrank_data = json.load(checkpoint_file)

dir_name = '_'.join([agg,alpha,ratio])
rank_factor = ['2','4','8','12','64','512']
lowrank_models = ['_'.join([model,factor]) for factor in rank_factor]

scaler_rate = ['0.64','0.5','0.4','0.3','0.1']

pruning_models = ['_'.join([model, scaler]) for scaler in scaler_rate]
def get_file_data(lowrank_model):
    try:
        file_list = sorted(os.listdir(os.path.join(root,dataset,lowrank_model,'_'.join([agg,alpha,ratio,warmup,dynamic]))))
        lowrank_file = open(os.path.join(root,dataset,lowrank_model,'_'.join([agg,alpha,ratio,warmup,dynamic]),file_list[-1],'0/log-1.json'))
    except:
        file_list = sorted(os.listdir(os.path.join(root,dataset,lowrank_model,'_'.join([agg,alpha,ratio,warmup,dynamic]))))
        lowrank_file = open(os.path.join(root,dataset,lowrank_model,'_'.join([agg,alpha,ratio,warmup,dynamic]),file_list[-1],'0/log-1.json'))
        
    data = json.load(lowrank_file)
    
    return data

lowrank_datas = [get_file_data(lowrank_models[i]) for i in range(len(lowrank_models))]
pruning_datas = [get_file_data(pruning_models[i]) for i in range(len(pruning_models))]

def draw_param_acc(length,path,
                   lowrank_datas,pruning_datas,fullrank_data, lowrank_params,
                   pruning_params,fullrank_param):
    plt.figure(dpi=300, figsize=(8,6))
    
    max_acc = 0
    min_acc = 100
    lowrank_perfs = []
    for data,param in zip(lowrank_datas, lowrank_params):
        print(len(data))
        best_acc = data[length * 2 - 1]['best_perfs']['top1'][0]
        lowrank_perfs.append(best_acc)
        print(best_acc)
        max_acc = max(max_acc, best_acc)
        min_acc = min(min_acc, best_acc)
    
    pruning_perfs = []
    for data, param in zip(pruning_datas, pruning_params):
        print(len(data))
        best_acc = data[length * 2 - 1]['best_perfs']['top1'][0]

        pruning_perfs.append(best_acc)
        print(best_acc)
        max_acc = max(max_acc, best_acc)
        min_acc = min(min_acc, best_acc)
        
    
    plt.plot(
        pruning_params,
        pruning_perfs,
        marker='s',
        label='Width Slimming Model',
        markersize=15,
        linewidth=4
    )
    
    plt.plot(
        lowrank_params,
        lowrank_perfs,
        marker='o',
        label='Low Rank Model',
        markersize=15,
        linewidth=4
    )
    
    fullrank_perf = datas[0][length * 2 - 1]['best_perfs']['top1'][0]
    plt.axhline(
        y=fullrank_perf,
        linestyle='-.',
        c='C2',
        label='Original Model',
        linewidth=4
    )
    
    min_acc = min(min_acc ,fullrank_perf)
    
    ax=plt.gca()
    plt.grid()
    x_major_locator=MultipleLocator(5)
    ax.yaxis.set_major_locator(x_major_locator)
    
    plt.legend(fontsize=25)
    plt.ylabel('Top-1 test accuracy',fontsize=30)
    plt.xlabel('Model Parameters(MB)',fontsize=30)
    plt.ylim(min_acc-2, max_acc + 3)
    plt.tick_params(labelsize=30)
    plt.savefig(path,bbox_inches='tight')
    plt.show()

lowrank_params=[8.40, 4.99, 3.27, 2.71, 1.78,0.29]
pruning_params=[8.77,5.35,3.42,1.94,0.22]

draw_param_acc(100, './pictures/cifar100_param.pdf',
               lowrank_datas,pruning_datas,fullrank_data, lowrank_params,
                  pruning_params,21.33)

In [ ]:
client_num = 10
target_acc = 65
colors = ['#386cb0','#d95f0e','#33a02c','#ca0020','#7b3294']
def draw_movement(length,path,names,datas,params):
    max_acc = 0
    plt.figure(dpi=300,figsize=(8,6))
    for name, data,param,color in zip(names,datas,params,colors):
        #datas = json.load(file)
        y = []
        comm_cost = 0
        #print(data)
        for i,file_data in enumerate(data):
            if (i % 2) == 0 and len(y) < length:
                y.append(file_data['top1'])
                if file_data['top1'] < target_acc:
                    comm_cost += 1
        print(name, comm_cost)
        comm_cost = comm_cost * param * client_num / 1024
        x = np.array(list(range(len(y))))*param*client_num*2 / 1024
        max_acc = max(np.max(y) + 1, max_acc)
        print(name,comm_cost)
        
        sns.lineplot(
            x = x ,
            y = y,
            legend = 'brief',
            label = name,
            ci = "sd",
            color=color,
            linewidth=2
        )
    plt.gcf()
    plt.grid()
    plt.legend(fontsize=20)
    plt.ylabel('Top-1 test accuracy',fontsize=30)
    plt.xlabel('Communication Cost(GB)',fontsize=30)
    min_acc = 20
    plt.ylim(min_acc, max_acc)
    plt.tick_params(labelsize=30)
    plt.savefig(path,bbox_inches='tight')
    plt.show()


flops = [1162.47,747.24, 633.99, 577.31, 557.9]
params = [21.33, 8.40, 4.99, 3.27, 2.71, 1.78,0.29]
draw_movement(100,'./pictures/cifar100_iid.pdf',
              ["Original Model","Low Rank Model(P.=8.40M)",'Low Rank Model(P.=4.99M)',"Low Rank Model(P.=3.27M)",\
                  "Low Rank Model(P.=2.71M)"],\
              iid_datas[:5],params[:5])

In [ ]:
x = 3
width= 0.18
macs = {
    'Conv Layer1':{'x':np.arange(x) + width,'y':[910.688,227.672, 151.781],'c':['#d7191c']*3,'h':'x'},
    'Conv Layer2':{'x':np.arange(x) + width*2,'y':[1144.259,286.065,134.676],'c':['#fdae61']*3,'h':'\\'},
    'Conv Layer3':{'x':np.arange(x) + width*3,'y':[1747.321,436.83,134.447],'c':['#a6d96a']*3,'h':'-'},
    'Conv Layer4':{'x':np.arange(x) + width*4,'y':[839.516,209.879,134.332],'c':['#1a9641']*3,'h':'+'},
    'FC Layer':{'x':np.arange(x) + width*5,'y':[0.102,0.051,0.005],'c':['#1a9850']*3,'h':'o'}
}

params = {
    'Conv Layer1':{'x':np.arange(x) + width,'y':[0.222,0.222,0.148],'c':['#d7191c']*3,'h':'x'},
    'Conv Layer2':{'x':np.arange(x) + width*2,'y':[1.116,1.116,0.526],'c':['#fdae61']*3,'h':'\\'},
    'Conv Layer3':{'x':np.arange(x) + width*3,'y':[6.822,6.822, 2.100],'c':['#a6d96a']*3,'h':'-'},
    'Conv Layer4':{'x':np.arange(x) + width*4,'y':[13.114,13.114,8.394],'c':['#1a9641']*3,'h':'+'},
    'FC Layer':{'x':np.arange(x) + width*5,'y':[0.103,0.051,0.005],'c':['#1a9850']*3,'h':'o'}
}

x_labels = list(['Tiny-ImageNet','CIFAR-100','CIFAR-10'])

fig, ax = plt.subplots(dpi=300,figsize=(10,6))
keys, values = list(macs.keys()),list(macs.values())

print(values)

for i in range(5):
    plt.bar(values[i]['x'],values[i]['y'],label=keys[i],width=width,hatch=values[i]['h'], 
            edgecolor='#5e3c99',
            color=values[i]['c'])
    
for value in values:
    xs, ys = value['x'],value['y']
    for i in range(len(xs)):
        plt.text(xs[i],ys[i]+1,'%.1f' %ys[i], ha='center', va= 'bottom',fontsize=14,rotation=35)

ax.set_xticks([i+0.3+width for i in range(x)])
ax.set_xticklabels(x_labels)
plt.ylim(0,2000)
plt.gcf()
plt.grid()
plt.ylabel('Number of MACs(MB)',fontsize=20)
plt.legend(fontsize=20,loc='upper right')
plt.tick_params(labelsize=20)
plt.savefig('./pictures/macs.pdf',bbox_inches='tight')

In [ ]:
x = 3
width= 0.18
macs = {
    'Conv Layer1':{'x':np.arange(x) + width,'y':[910.688,227.672, 151.781],'c':['#d7191c']*3,'h':'x'},
    'Conv Layer2':{'x':np.arange(x) + width*2,'y':[1144.259,286.065,134.676],'c':['#fdae61']*3,'h':'\\'},
    'Conv Layer3':{'x':np.arange(x) + width*3,'y':[1747.321,436.83,134.447],'c':['#a6d96a']*3,'h':'-'},
    'Conv Layer4':{'x':np.arange(x) + width*4,'y':[839.516,209.879,134.332],'c':['#1a9641']*3,'h':'+'},
    'FC Layer':{'x':np.arange(x) + width*5,'y':[0.102,0.051,0.005],'c':['#1a9850']*3,'h':'o'}
}

params = {
    'Conv Layer1':{'x':np.arange(x) + width,'y':[0.222,0.222,0.148],'c':['#d7191c']*3,'h':'x'},
    'Conv Layer2':{'x':np.arange(x) + width*2,'y':[1.116,1.116,0.526],'c':['#fdae61']*3,'h':'\\'},
    'Conv Layer3':{'x':np.arange(x) + width*3,'y':[6.822,6.822, 2.100],'c':['#a6d96a']*3,'h':'-'},
    'Conv Layer4':{'x':np.arange(x) + width*4,'y':[13.114,13.114,8.394],'c':['#1a9641']*3,'h':'+'},
    'FC Layer':{'x':np.arange(x) + width*5,'y':[0.103,0.051,0.005],'c':['#1a9850']*3,'h':'o'}
}

x_labels = list(['Tiny-ImageNet','CIFAR-100','CIFAR-10'])

fig, ax = plt.subplots(dpi=300,figsize=(10,6))
keys, values = list(macs.keys()),list(macs.values())

print(values)

for i in range(5):
    plt.bar(values[i]['x'],values[i]['y'],label=keys[i],width=width,hatch=values[i]['h'], 
            edgelor='#5e3c99',
            color=values[i]['c'])
    
for value in values:
    xs, ys = value['x'],value['y']
    for i in range(len(xs)):
        plt.text(xs[i],ys[i]+1,'%.1f' %ys[i], ha='center', va= 'bottom',fontsize=14,rotation=35)

ax.set_xticks([i+0.3+width for i in range(x)])
ax.set_xticklabels(x_labels)
plt.ylim(0,2000)
plt.gcf()
plt.grid()
plt.ylabel('Number of MACs(MB)',fontsize=20)
plt.legend(fontsize=20,loc='upper right')
plt.tick_params(labelsize=20)
plt.savefig('./pictures/macs.pdf',bbox_inches='tight')

In [ ]:
pruning_model = "resnet34_1:resnet34_0.64:resnet34_0.5:resnet34_0.4"
agg = "pruning"
dir_name = '_'.join([agg,alpha,ratio,warmup,dynamic])#,warmup,unit

file_list = sorted(os.listdir(os.path.join(root,dataset,pruning_model,dir_name)))
checkpoint_file = open(os.path.join(root,dataset,pruning_model,dir_name,file_list[-1],'0/log-1.json'))
hete_pruning_data = json.load(checkpoint_file)
print(hete_pruning_data)

In [ ]:
import matplotlib.pyplot as plt
pruning_target_name = 'aggregated_test_loader_resnet34_0.4-0'
lowrank_target_name = 'aggregated_test_loader_resnet34_12-0'
large_target_name = 'aggregated_test_loader-0'

client_num = 10
linestyles =['--','-','-.']
def draw_movement(path,labels,names,file_datas,params):
    max_acc = 0
    plt.figure(dpi=300,figsize=(8,6))
    
    for label,name, datas,param,linestyle in zip(labels,names,file_datas,params,linestyles):
        y = []
        comm_cost = 0
        for i,data in enumerate(datas):
            if data['type'] == name:
                y.append(data['top1'])
                    
        x = np.array(list(range(len(y))))
        
        max_acc = max(np.max(y) + 1, max_acc)
        sns.lineplot(
            x = x ,
            y = y,
            legend = 'brief',
            label = label,
            ci = "sd",
            linewidth=2,
            linestyle=linestyle
        )
    ax = plt.gca()
    x_major_locator=MultipleLocator(25)
    ax.xaxis.set_major_locator(x_major_locator)
    plt.grid()
    plt.legend(fontsize=25)
    plt.ylabel('Top-1 test accuracy',fontsize=30)
    plt.xlabel('# of communication rounds',fontsize=30)
    min_acc = 20
    plt.ylim(min_acc, max_acc)
    plt.tick_params(labelsize=30)
    plt.savefig(path,bbox_inches='tight')
    plt.show()
    
draw_movement('./pictures/cifar100_compare_iid.pdf',
              ['HeteroFL','FedHM','Low Rank Model'],\
              [pruning_target_name,lowrank_target_name,large_target_name],\
              [hete_pruning_data,hete_lowrank_data,homo_lowrank_data],\
              [3.42,2.71,2.71])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from matplotlib.pyplot import MultipleLocator
import os
root = './data/checkpoint/'
dataset = "cifar100"
model = "resnet34"
fullrank_model = "_".join([model,"1"])
agg = "fedavg"
alpha = 'alpha0'
ratio = 'ratio0.5'

warmup = 'warmup0
dynamic='dynamicFalse'
dir_name = '_'.join([agg,alpha,ratio,warmup,dynamic])#,warmup,unit
print(dir_name)

dir_name = '_'.join([agg,alpha,ratio,warmup,dynamic])#,warmup,
print(dir_name)

file_list = sorted(os.listdir(os.path.join(root,dataset,fullrank_model,dir_name)))
checkpoint_file = open(os.path.join(root,dataset,fullrank_model,dir_name,file_list[-1],'0/log-1.json'))
fullrank_data = json.load(checkpoint_file)

dir_name = '_'.join([agg,alpha,ratio])
rank_factor = ['2','4','8','12','64','512']
lowrank_models = ['_'.join([model,factor]) for factor in rank_factor]

scaler_rate = ['0.64','0.5','0.4','0.3','0.1']

pruning_models = ['_'.join([model, scaler]) for scaler in scaler_rate]
def get_file_data(lowrank_model):
    try:
        file_list = sorted(os.listdir(os.path.join(root,dataset,lowrank_model,'_'.join([agg,alpha,ratio,warmup,dynamic]))))
        lowrank_file = open(os.path.join(root,dataset,lowrank_model,'_'.join([agg,alpha,ratio,warmup,dynamic]),file_list[-1],'0/log-1.json'))
    except:
        file_list = sorted(os.listdir(os.path.join(root,dataset,lowrank_model,'_'.join([agg,alpha,ratio,warmup,dynamic]))))
        lowrank_file = open(os.path.join(root,dataset,lowrank_model,'_'.join([agg,alpha,ratio,warmup,dynamic]),file_list[-1],'0/log-1.json'))
        
    data = json.load(lowrank_file)
    
    return data

lowrank_datas = [get_file_data(lowrank_models[i]) for i in range(len(lowrank_models))]
pruning_datas = [get_file_data(pruning_models[i]) for i in range(len(pruning_models))]

def draw_param_acc(length,path,
                   lowrank_datas,pruning_datas,fullrank_data, lowrank_params,
                   pruning_params,fullrank_param):
    plt.figure(dpi=300, figsize=(8,6))
    
    max_acc = 0
    min_acc = 100
    lowrank_perfs = []
    for data,param in zip(lowrank_datas, lowrank_params):
        print(len(data))
        best_acc = data[length * 2 - 1]['best_perfs']['top1'][0]
        lowrank_perfs.append(best_acc)
        print(best_acc)
        max_acc = max(max_acc, best_acc)
        min_acc = min(min_acc, best_acc)
    
    pruning_perfs = []
    for data, param in zip(pruning_datas, pruning_params):
        print(len(data))
        best_acc = data[length * 2 - 1]['best_perfs']['top1'][0]

        pruning_perfs.append(best_acc)
        print(best_acc)
        max_acc = max(max_acc, best_acc)
        min_acc = min(min_acc, best_acc)
        
    
    plt.plot(
        pruning_params,
        pruning_perfs,
        marker='s',
        label='Width Slimming Model',
        markersize=15,
        linewidth=4
    )
    
    plt.plot(
        lowrank_params,
        lowrank_perfs,
        marker='o',
        label='Low Rank Model',
        markersize=15,
        linewidth=4
    )
    
    fullrank_perf = datas[0][length * 2 - 1]['best_perfs']['top1'][0]
    plt.axhline(
        y=fullrank_perf,
        linestyle='-.',
        c='C2',
        label='Original Model',
        linewidth=4
    )
    
    min_acc = min(min_acc ,fullrank_perf)
    
    ax=plt.gca()
    plt.grid()
    x_major_locator=MultipleLocator(5)
    ax.yaxis.set_major_locator(x_major_locator)
    
    plt.legend(fontsize=25)
    plt.ylabel('Top-1 test accuracy',fontsize=30)
    plt.xlabel('Model Parameters(MB)',fontsize=30)
    plt.ylim(min_acc-2, max_acc + 3)
    plt.tick_params(labelsize=30)
    plt.savefig(path,bbox_inches='tight')
    plt.show()

lowrank_params=[8.40, 4.99, 3.27, 2.71, 1.78,0.29]
pruning_params=[8.77,5.35,3.42,1.94,0.22]

draw_param_acc(100, './pictures/cifar100_param.pdf',
               lowrank_datas,pruning_datas,fullrank_data, lowrank_params,
                  pruning_params,21.33)

In [ ]:
client_num = 10
target_acc = 65
colors = ['#386cb0','#d95f0e','#33a02c','#ca0020','#7b3294']
def draw_movement(length,path,names,datas,params):
    max_acc = 0
    plt.figure(dpi=300,figsize=(8,6))
    for name, data,param,color in zip(names,datas,params,colors):
        #datas = json.load(file)
        y = []
        comm_cost = 0
        #print(data)
        for i,file_data in enumerate(data):
            if (i % 2) == 0 and len(y) < length:
                y.append(file_data['top1'])
                if file_data['top1'] < target_acc:
                    comm_cost += 1
        print(name, comm_cost)
        comm_cost = comm_cost * param * client_num / 1024
        x = np.array(list(range(len(y))))*param*client_num*2 / 1024
        max_acc = max(np.max(y) + 1, max_acc)
        print(name,comm_cost)
        
        sns.lineplot(
            x = x ,
            y = y,
            legend = 'brief',
            label = name,
            ci = "sd",
            color=color,
            linewidth=2
        )
    plt.gcf()
    plt.grid()
    plt.legend(fontsize=20)
    plt.ylabel('Top-1 test accuracy',fontsize=30)
    plt.xlabel('Communication Cost(GB)',fontsize=30)
    min_acc = 20
    plt.ylim(min_acc, max_acc)
    plt.tick_params(labelsize=30)
    plt.savefig(path,bbox_inches='tight')
    plt.show()


flops = [1162.47,747.24, 633.99, 577.31, 557.9]
params = [21.33, 8.40, 4.99, 3.27, 2.71, 1.78,0.29]
draw_movement(100,'./pictures/cifar100_iid.pdf',
              ["Original Model","Low Rank Model(P.=8.40M)",'Low Rank Model(P.=4.99M)',"Low Rank Model(P.=3.27M)",\
                  "Low Rank Model(P.=2.71M)"],\
              iid_datas[:5],params[:5])

In [ ]:
x = 3
width= 0.18
macs = {
    'Conv Layer1':{'x':np.arange(x) + width,'y':[910.688,227.672, 151.781],'c':['#d7191c']*3,'h':'x'},
    'Conv Layer2':{'x':np.arange(x) + width*2,'y':[1144.259,286.065,134.676],'c':['#fdae61']*3,'h':'\\'},
    'Conv Layer3':{'x':np.arange(x) + width*3,'y':[1747.321,436.83,134.447],'c':['#a6d96a']*3,'h':'-'},
    'Conv Layer4':{'x':np.arange(x) + width*4,'y':[839.516,209.879,134.332],'c':['#1a9641']*3,'h':'+'},
    'FC Layer':{'x':np.arange(x) + width*5,'y':[0.102,0.051,0.005],'c':['#1a9850']*3,'h':'o'}
}

params = {
    'Conv Layer1':{'x':np.arange(x) + width,'y':[0.222,0.222,0.148],'c':['#d7191c']*3,'h':'x'},
    'Conv Layer2':{'x':np.arange(x) + width*2,'y':[1.116,1.116,0.526],'c':['#fdae61']*3,'h':'\\'},
    'Conv Layer3':{'x':np.arange(x) + width*3,'y':[6.822,6.822, 2.100],'c':['#a6d96a']*3,'h':'-'},
    'Conv Layer4':{'x':np.arange(x) + width*4,'y':[13.114,13.114,8.394],'c':['#1a9641']*3,'h':'+'},
    'FC Layer':{'x':np.arange(x) + width*5,'y':[0.103,0.051,0.005],'c':['#1a9850']*3,'h':'o'}
}


x_labels = list(['Tiny-ImageNet','CIFAR-100','CIFAR-10'])

fig, ax = plt.subplots(dpi=300,figsize=(10,6))
keys, values = list(macs.keys()),list(macs.values())

print(values)

for i in range(5):
    plt.bar(values[i]['x'],values[i]['y'],label=keys[i],width=width,hatch=values[i]['h'], 
            edgecolor='#5e3c99',
            color=values[i]['c'])
    
for value in values:
    xs, ys = value['x'],value['y']
    for i in range(len(xs)):
        plt.text(xs[i],ys[i]+1,'%.1f' %ys[i], ha='center', va= 'bottom',fontsize=14,rotation=35)

ax.set_xticks([i+0.3+width for i in range(x)])
ax.set_xticklabels(x_labels)
plt.ylim(0,2000)
plt.gcf()
plt.grid()
plt.ylabel('Number of MACs(MB)',fontsize=20)
plt.legend(fontsize=20,loc='upper right')
plt.tick_params(labelsize=20)
plt.savefig('./pictures/macs.pdf',bbox_inches='tight')